In [27]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import random as rnd

from sklearn.model_selection import cross_val_score

In [28]:
#raw = pd.read_csv('data/2007flightrecords.csv')
raw = pd.read_csv('http://stat-computing.org/dataexpo/2009/2008.csv.bz2')

#Drop all cancelled flights
raw = raw[raw['Cancelled'] == 0]
raw.drop(columns = ['Cancelled'], inplace = True)

raw.head()

,Year,Month,DayofMonth,DayOfWeek,DepTime,CRSDepTime,ArrTime,CRSArrTime,UniqueCarrier,FlightNum,...,Distance,TaxiIn,TaxiOut,CancellationCode,Diverted,CarrierDelay,WeatherDelay,NASDelay,SecurityDelay,LateAircraftDelay
0,2008,1,3,4,2003.0,1955,2211.0,2225,WN,335,...,810,4.0,8.0,NaN,0,NaN,NaN,NaN,NaN,NaN
1,2008,1,3,4,754.0,735,1002.0,1000,WN,3231,...,810,5.0,10.0,NaN,0,NaN,NaN,NaN,NaN,NaN
2,2008,1,3,4,628.0,620,804.0,750,WN,448,...,515,3.0,17.0,NaN,0,NaN,NaN,NaN,NaN,NaN
3,2008,1,3,4,926.0,930,1054.0,1100,WN,1746,...,515,3.0,7.0,NaN,0,NaN,NaN,NaN,NaN,NaN
4,2008,1,3,4,1829.0,1755,1959.0,1925,WN,3920,...,515,3.0,10.0,NaN,0,2.0,0.0,0.0,0.0,32.0


In [12]:
raw_samp = raw.sample(n = 1000)#raw.iloc[:1000] #Take a small sample to test row manipulation

#Year column is about as useful as a bathing suit in a nunery, but I will select my features later.

In [13]:
print(raw_samp.columns)
print(raw_samp['UniqueCarrier'].unique())

Index(['Year', 'Month', 'DayofMonth', 'DayOfWeek', 'DepTime', 'CRSDepTime',
       'ArrTime', 'CRSArrTime', 'UniqueCarrier', 'FlightNum', 'TailNum',
       'ActualElapsedTime', 'CRSElapsedTime', 'AirTime', 'ArrDelay',
       'DepDelay', 'Origin', 'Dest', 'Distance', 'TaxiIn', 'TaxiOut',
       'CancellationCode', 'Diverted', 'CarrierDelay', 'WeatherDelay',
       'NASDelay', 'SecurityDelay', 'LateAircraftDelay'],
      dtype='object')
['AS' 'EV' 'AA' 'OH' 'MQ' 'XE' 'US' 'YV' 'WN' 'OO' 'NW' 'CO' 'UA' 'F9'
 'FL' 'DL' '9E' 'HA' 'B6' 'AQ']


In [14]:
#Set up outcome. Is it more than 30 minutes late?
#raw_samp[['DepTime', 'CRSDepTime','ArrTime', 'CRSArrTime', 'ArrDelay']]

#Luckily, we have the ArrDelay already
#All we have to do is:
raw_samp['Late'] = np.where(raw_samp['ArrDelay'] >= 30, 1, 0)
#print(raw_samp['Late'])

#Create binary feature for the 'UniqueCarrier' columns
for carrier in raw_samp['UniqueCarrier'].unique():
    raw_samp[str(carrier)] = np.where(raw_samp['UniqueCarrier'] == carrier, 1, 0)
    
print(raw_samp.columns)

Index(['Year', 'Month', 'DayofMonth', 'DayOfWeek', 'DepTime', 'CRSDepTime',
       'ArrTime', 'CRSArrTime', 'UniqueCarrier', 'FlightNum', 'TailNum',
       'ActualElapsedTime', 'CRSElapsedTime', 'AirTime', 'ArrDelay',
       'DepDelay', 'Origin', 'Dest', 'Distance', 'TaxiIn', 'TaxiOut',
       'CancellationCode', 'Diverted', 'CarrierDelay', 'WeatherDelay',
       'NASDelay', 'SecurityDelay', 'LateAircraftDelay', 'Late', 'AS', 'EV',
       'AA', 'OH', 'MQ', 'XE', 'US', 'YV', 'WN', 'OO', 'NW', 'CO', 'UA', 'F9',
       'FL', 'DL', '9E', 'HA', 'B6', 'AQ'],
      dtype='object')


In [22]:
#Set up features and outcomes for the sample
Y_samp = raw_samp['Late']

carriers = np.asarray(raw_samp['UniqueCarrier'].unique())
print(type(carriers))
X_samp = raw_samp[['Month', 'DayofMonth', 'DayOfWeek', 'Distance', 'AS', 'EV',
       'AA', 'OH', 'MQ', 'XE', 'US', 'YV', 'WN', 'OO', 'NW', 'CO', 'UA', 'F9',
       'FL', 'DL', '9E', 'HA', 'B6', 'AQ']]

X_samp.head()

<class 'numpy.ndarray'>


,Month,DayofMonth,DayOfWeek,Distance,AS,EV,AA,OH,MQ,XE,...,NW,CO,UA,F9,FL,DL,9E,HA,B6,AQ
2919342,5,1,4,123,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3953490,7,20,7,564,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
5868869,10,28,2,867,0,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3766775,7,21,1,280,0,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0
5144857,9,19,5,453,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [32]:
#Lets try KNN
from sklearn.neighbors import KNeighborsClassifier

neighbors = KNeighborsClassifier(n_neighbors = 10)

cross_val_score(neighbors, X_samp, Y_samp, cv = 10)

array([0.87128713, 0.87128713, 0.87128713, 0.87      , 0.87      ,
       0.87      , 0.87      , 0.87878788, 0.87878788, 0.87878788])

# Training on a random sample of the raw data (n = 1000)

__KNN Scores, n_neighbors = 1__

Even with no optimization and only looking at each point's nearest neighbor, we get halfway decent scores.

[0.8019802 , 0.75247525, 0.81188119, 0.75, 0.77 ,0.81, 0.75, 0.84848485, 0.75757576, 0.83838384]

But we can do a lot better.

__KNN Scores, n_neighbors = 10__

Things get better with a magnitude increase in neighbors. We get better values and higher scores.

[0.87128713, 0.87128713, 0.87128713, 0.87, 0.87, 0.87, 0.87, 0.87878788, 0.87878788, 0.87878788]

